In [1]:
import pandas as pd
import numpy as np
import tarfile

## Parse data from a day's worth of live updates

In [2]:
RAW_DATA_LOCATION = "data/raw/"
PARSED_DATA_LOCATION = "data/parsed/"

In [4]:
day = "2017-10-10"

live_data = pd.DataFrame()

# This will run through each file in the tar.gz archive and parse it's data
tar = tarfile.open(RAW_DATA_LOCATION + day + ".tar.gz", "r:gz")
for member in tar.getmembers():
    extracted_file = tar.extractfile(member)
    if extracted_file:
        content = extracted_file.read()
        
        # Parse data for this minute's update file
        temp = pd.DataFrame.from_records(pd.read_json(content, orient='index')["VP"].values)
        temp['timestamp'] = member.name.split('/')[1].strip('.json')
        
        # Add it to the main dataframe
        live_data = pd.concat([live_data, temp])

In [5]:
# Save the new dataframe as a compressed csv
# Compressing ensures the files are a managable size for version control
live_data.to_csv(PARSED_DATA_LOCATION + day + ".csv.gz", index=False, compression='gzip')

## Reading a parsed file

In [12]:
# Read the data just like any other CSV. Pandas will automatically handle the decompression
parsed_data = pd.read_csv("data/parsed/2017-10-10.csv.gz")

parsed_data.head()

,desi,dir,dl,hdg,jrn,lat,line,long,oday,odo,oper,source,spd,start,stop_index,timestamp,tsi,tst,veh
0,106,2.0,-8,NaN,XXX,60.217000,XXX,24.82504,2017-10-11,NaN,XXX,hsl roisto,0.0,0016,NaN,1507670281,1507670272,2017-10-10T21:17:52.000Z,01a28423
1,102T,1.0,119,NaN,XXX,60.166460,XXX,24.84158,2017-10-11,NaN,XXX,hsl roisto,0.0,0005,NaN,1507670281,1507670219,2017-10-10T21:16:59.000Z,023d7289
2,236,1.0,-7,NaN,XXX,60.267020,XXX,24.74761,2017-10-10,NaN,XXX,hsl roisto,0.0,2353,NaN,1507670281,1507670213,2017-10-10T21:16:53.000Z,02d84a34
3,560,1.0,-111,NaN,XXX,60.242531,XXX,25.01917,2017-10-10,NaN,XXX,hsl roisto,0.0,2350,NaN,1507670281,1507670229,2017-10-10T21:17:09.000Z,0422f230
4,18,1.0,-11,NaN,XXX,60.200160,XXX,24.68547,2017-10-10,NaN,XXX,hsl roisto,0.0,2356,NaN,1507670281,1507670269,2017-10-10T21:17:49.000Z,0722170c
